In [1]:
!pip install transformers
!pip install seqeval
!pip install tensorboardx

You should consider upgrading via the '/Users/patsnap/anaconda2/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/patsnap/anaconda2/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/patsnap/anaconda2/bin/python -m pip install --upgrade pip' command.


In [2]:
!pip install simpletransformers

You should consider upgrading via the '/Users/patsnap/anaconda2/bin/python -m pip install --upgrade pip' command.


In [4]:
import pandas as pd
train_df = pd.read_csv('/Users/patsnap/Desktop/Neo4J_and_other_codes/text_classification/jigsaw-toxic-comment-classification-challenge/train.csv')
train_df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [6]:
df = pd.read_csv('/Users/patsnap/Desktop/Neo4J_and_other_codes/text_classification/jigsaw-toxic-comment-classification-challenge/train.csv')
df.head()
df['labels'] = list(zip(df.toxic.tolist(), df.severe_toxic.tolist(), df.obscene.tolist(), df.threat.tolist(),  df.insult.tolist(), df.identity_hate.tolist()))
df['text'] = df['comment_text'].apply(lambda x: x.replace('\n', ' '))

df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,labels,text
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)",Explanation Why the edits made under my userna...
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)",D'aww! He matches this background colour I'm s...
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)","Hey man, I'm really not trying to edit war. It..."
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)",""" More I can't make any real suggestions on im..."
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,"(0, 0, 0, 0, 0, 0)","You, sir, are my hero. Any chance you remember..."


In [8]:
from sklearn.model_selection import train_test_split
train_df, eval_df = train_test_split(df, test_size=0.2)

In [11]:
from simpletransformers.classification import MultiLabelClassificationModel
model = MultiLabelClassificationModel('roberta', 'roberta-base', use_cuda = False, num_labels=6, args={'train_batch_size':2, 'gradient_accumulation_steps':16, 'learning_rate': 3e-5, 'num_train_epochs': 3, 'max_seq_length': 512})

# Default arguments - MultiClass classification
self.args = {
    "output_dir": "outputs/",
    "cache_dir": "cache_dir/",

    "fp16": True,
    "fp16_opt_level": "O1",
    "max_seq_length": 128,
    "train_batch_size": 8,
    "gradient_accumulation_steps": 1,
    "eval_batch_size": 8,
    "num_train_epochs": 1,
    "weight_decay": 0,
    "learning_rate": 4e-5,
    "adam_epsilon": 1e-8,
    "warmup_ratio": 0.06,
    "warmup_steps": 0,
    "max_grad_norm": 1.0,

    "logging_steps": 50,
    "save_steps": 2000,

    "overwrite_output_dir": False,
    "reprocess_input_data": False,
    "evaluate_during_training": False,

    "process_count": cpu_count() - 2 if cpu_count() > 2 else 1,
    "n_gpu": 1,
}

# To load a model a previously saved model instead of a default model
model = MultiLabelClassificationModel('xlnet', 'path_to_model/', num_labels=6)

In [12]:
# Train the model
model.train_model(train_df)

Running loss: 0.722954

KeyboardInterrupt: 

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

In [ ]:
test_df = pd.read_csv('/Users/patsnap/Desktop/Neo4J_and_other_codes/text_classification/jigsaw-toxic-comment-classification-challenge/test.csv')

to_predict = test_df.comment_text.apply(lambda x: x.replace('\n', ' ')).tolist()
preds, outputs = model.predict(to_predict)

sub_df = pd.DataFrame(outputs, columns=['toxic','severe_toxic','obscene','threat','insult','identity_hate'])


sub_df['id'] = test_df['id']
sub_df = sub_df[['id', 'toxic','severe_toxic','obscene','threat','insult','identity_hate']]